### Relative energy function

In [12]:
#function that transcribe energy value into relative energy value 
def energy_rela(df):
    if 'energy' in df.columns and 'no_nickel' in df.columns and 'joining_type' in df.columns:
        print('Appriopiate columns are present in database')
        df['rela_energy'] = df['no_nickel']
        for i in range(len(df)):
            energy = -df['energy'].iloc[i]
            nickel_part = monomet_cl_energy[df['joining_type'].iloc[i]][0]
            copper_part = monomet_cl_energy[df['joining_type'].iloc[i]][1]
            nickel = int(df['no_nickel'].iloc[i])           
            df['rela_energy'].iloc[i] = (energy - (1/13)*(nickel*nickel_part + (13-nickel)*copper_part))
        return df
    else:
        print('Given database is missing at least one column')
        


### Proper data preprocessing

In [19]:
import pandas as pd
from p9_sp_clusters_data import *
#loading data
simplify_cluster_df = pd.DataFrame(data=list_of_sp_clusters, columns = ['full_notation','energy','SCF_steps','magnetism','time_of_calculation'])
all_csv = pd.read_csv('all.csv').drop(['Unnamed: 0','all_corners_comb','all_edges_comb','no_edges_comb','no_corners_comb','all_comb'],axis=1)
benchmark = pd.read_csv('benchmark_ds.csv').drop(['index','Unnamed: 7'],axis=1)

#simplify_cluster_data is a dataset containing the energy information of the clusters determined by me according to the proposed simplified methodology
#all_csv is a dataset containing comprehensive information about each individual metallic cluster
#benchmark is a dataset containing information about clusters calculated very accurately by a PhD student


### Working on simplify_cluster_data

In [33]:
#extract information about docking 
simplify_cluster_df = simplify_cluster_df.assign(number = simplify_cluster_df['full_notation'].apply(lambda x: x.split('_')[0]),
              joining_type = simplify_cluster_df['full_notation'].apply(lambda x: x.split('_')[1]),
              first_atom = simplify_cluster_df['full_notation'].apply(lambda x: x.split('_')[2]),
              second_atom = simplify_cluster_df['full_notation'].apply(lambda x: x.split('_')[3]),
              energy = simplify_cluster_df['energy'].apply(lambda x: float(x)),
              no_nickel = all_csv['no_nickel'])
              
#hydrogen correction
for cluster in range(len(simplify_cluster_df)):
    if simplify_cluster_df['joining_type'].iloc[cluster] in ['MOCOM','MOCMHnbond','MC']:
        simplify_cluster_df['energy'].iloc[cluster] -= 3.344
        
#gathering data from all.csv file and splitting shortest paths, conformation and bonds
for column in ['ni_ncentre','ni_centre','cu_ncentre','cu_centre','0ni','8ni','17ni','25ni','33ni','42ni','50ni', 'nini','cucu','nicu','type_of_first_atom','type_of_second_atom', 'hydrogen']:
    df[column] = df['magnetism']  
    
for cluster in range(len(simplify_cluster_df)):
    for column in ['no_nickel','c_atom','mass_center','shortest_paths','conformation','bonds']:
        simplify_cluster_df[column].iloc[cluster] = all_csv[column].loc[int(simplify_cluster_df['number'].iloc[cluster])]
    for column in ['topology']:
        simplify_cluster_df[column].iloc[cluster] = topology_to_list(all_csv[column].loc[int(simplify_cluster_df['number'].iloc[cluster])])


/tmp/ipykernel_9087/3778803193.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  simplify_cluster_df['energy'].iloc[cluster] -= 3.344


KeyError: 'no_nickel'

In [30]:
df

,full_notation,energy,SCF_steps,magnetism,time_of_calculation
0,58_MO_7_11,-65.233019,94,0.95,67.92m
1,58_MOCOM_7_11,-70.411401,118,2.00,113.04m
2,59_MO_1_6,-65.574785,109,1.00,79.86m
3,59_MO_4_1,-65.529727,88,0.99,43.05m
4,59_MO_7_6,-65.504183,86,1.00,56.84m
...,...,...,...,...,...
644,141_MC_11_6,-70.261721,42,4.00,24.01m
645,156_MC_5_4,-71.162286,51,4.00,36.18m
646,190_MOCM_8_7,-73.355261,43,7.00,23.71m
647,150_MOCMHnbond_5_10,-71.674709,45,4.00,26.44m
